tar -C /tmp/ayu -zvxf xyz.tar.gz

In [1]:
import os,sys,pickle,time
import pandas as pd
import numpy as np
import subprocess
from multiprocessing import Pool 
import shutil
from tqdm import tqdm

In [6]:
tgz_dir = "../RAID5/pcap/"
decompress_dir = "../data_hdd4t/pcap_process/"
all_dates = next(os.walk(tgz_dir))[1]
all_dates = sorted(all_dates)
all_dates

['2019_12_29',
 '2019_12_30',
 '2019_12_31',
 '2020_01_01',
 '2020_01_02',
 '2020_01_03',
 '2020_01_04',
 '2020_01_05',
 '2020_01_06',
 '2020_01_07',
 '2020_01_08',
 '2020_01_09',
 '2020_01_10',
 '2020_01_11',
 '2020_01_12',
 '2020_01_13',
 '2020_01_14',
 '2020_01_15',
 '2020_01_16',
 '2020_01_17',
 '2020_01_18',
 '2020_01_19',
 '2020_01_20',
 '2020_01_21',
 '2020_01_22',
 '2020_01_23',
 '2020_01_24',
 '2020_01_25',
 '2020_01_26',
 '2020_01_27',
 '2020_01_28',
 '2020_01_29',
 '2020_01_30',
 '2020_01_31',
 '2020_02_01',
 '2020_02_02',
 '2020_02_03',
 '2020_02_04',
 '2020_02_05',
 '2020_02_06',
 '2020_02_07',
 '2020_02_08']

In [36]:
def parallel_untar(in_file,out_dir):
    test_dir = out_dir.split('/')
    test_dir = test_dir[:3]
    test_dir = "/".join(test_dir)
    total, used, free = shutil.disk_usage(test_dir)
    if (free<total*0.05) or (free // (2**30) < 130):
#         cmd = ['rmdir',out_dir]
#         p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
#         out, err = p.communicate()
#         print("!Error:", err, '!')
        print('Disk not enough space ERR:',in_file,out_dir)
        return out_dir
    else:
        if not os.path.exists(out_dir):
            os.makedirs(out_dir, exist_ok=True)
        cmd = ["tar","-C", out_dir,
           "-zvxf", in_file]
        p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
        out, err = p.communicate()
        print("!Error:", err, '! Done:',out_dir)
    

## For ISP's Pcap
parallel by function name

In [22]:
# %%time
in_file_paths = []
out_dir_paths = []
days = ['2019_12_31','2020_01_01','2020_01_02','2020_01_30','2020_01_31','2020_02_01'] #指定日期，清空掉就是全部日子
isps = ['中華電信']
for date in tqdm(all_dates):
    if date not in days:
        continue
    pcap_path = tgz_dir + date + '/'
    isp_names = next(os.walk(pcap_path))[1]
    take_isps = []
    for isp in isp_names:
        if isp in isps:
            take_isps.append(isp)
    for isp in take_isps:
        file_names = next(os.walk(pcap_path+isp))[2]
        file_names = list(filter(lambda f: f.endswith(".tar.gz"), file_names))
        if len(file_names)>1:
            print('WARNING: more than 1 pcap tgz file in',pcap_path+isp+'/')
        for file_name in file_names:
            pcap_path_ = pcap_path+isp+'/'+file_name
            output_path = decompress_dir+date+'/'+isp+'/'+file_name.replace('.tar.gz','_dir')+'/'
            in_file_paths.append(pcap_path_)
            out_dir_paths.append(output_path)

assert len(in_file_paths) == len(out_dir_paths)
assert in_file_paths[0].split('/')[3] == out_dir_paths[0].split('/')[3]
assert in_file_paths[-1].split('/')[4] == out_dir_paths[-1].split('/')[4]
len(in_file_paths)

100%|██████████| 42/42 [00:00<00:00, 25086.98it/s]


6

In [ ]:
# #create new dir for decompress
# pcap_path = root_dir + name + '/pcap/'
# file_names = next(os.walk(pcap_path))[2]
# file_names = list(filter(lambda f: f.endswith(".tar.gz"), file_names))
# new_dir_names = [x.replace('.tar.gz','_dir') for x in file_names]
# for dir_name in new_dir_names:
#     path = pcap_path+dir_name
#     try:
#         if days != []:
#             for day in days:
#                 if day in path:
#                     os.makedirs(path, exist_ok=True)
#                     paths.append(path)
#         else:
#             os.makedirs(path, exist_ok=True)
#             paths.append(path)
#     except NameError:
#         os.makedirs(path, exist_ok=True)
#         paths.append(path)

# paths

In [37]:
%%time
load_start = time.time()
pool_result = []
pool = Pool(processes=2) #processes=21
for path_in,dir_out in zip(in_file_paths,out_dir_paths):
    r = pool.apply_async(parallel_untar,args=(path_in,dir_out))
    pool_result.append(r)
pool.close()
pool.join()
for r in pool_result:
    print('return:',r.get())
load_end = time.time() - load_start
print("Decompress Time:", '{:02f}:{:02f}:{:02f}'.format(load_end // 3600, (load_end % 3600 // 60), load_end % 60))

!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
return: None
return: None
return: None
return: None
return: None
return: None
Decompress Time: 1.000000:30.000000:31.162456
CPU times: user 2.85 s, sys: 813 ms, total: 3.66 s
Wall time: 1h 30min 31s


In [28]:
r.get()

FileNotFoundError: [Errno 2] No such file or directory: '../data_hdd4t/pcap_process/2020_02_01/中華電信/pcap_2020_02_01_dir/'

In [29]:
not os.path.exists(dir_out)

True

### remove folder recursively

In [34]:
for name in tqdm(company_names):
#     if name == '中華電信': #指定電信公司
#         continue
    pcap_path = root_dir + name + '/pcap/'
    dir_names = next(os.walk(pcap_path))[1]
    if dir_names != []:
        for folder in dir_names:
            shutil.rmtree(pcap_path+folder+'/')
            print(pcap_path+folder+'/')

  0%|          | 0/11 [00:00<?, ?it/s]

../data_hdd4t/中華電信/pcap/snort.2020-01-04_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-10_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-09_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-02_dir/


100%|██████████| 11/11 [00:04<00:00,  2.25it/s]

../data_hdd4t/中華電信/pcap/snort.2020-01-11_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-08_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-05_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-06_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-07_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-03_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-01_dir/


## For ISPs' honeypot

In [22]:
paths = []
days = ['2020-01-09','2020-01-10','2020-01-11'] #指定日期，清空掉就是全部日子
for name in tqdm(company_names):
    if name != '中華電信': #指定電信公司，註解掉就是全部
        continue
#     print('=== Running:',name,'===')
    hp_path = root_dir + name + '/honeypot/'
    date_dir_names = next(os.walk(hp_path))[1]
    if days != []:
        temp = []
        for date in date_dir_names:
            for day in days:
                if day == date:
                    temp.append(date)
        date_dir_names = temp
    else:
        pass
    for date in date_dir_names:
        hp_date_path = hp_path + date +'/'
        hp_names_dir = next(os.walk(hp_date_path))[1]
        for name in hp_names_dir:
            file_names = next(os.walk(hp_date_path+name+'/'))[2]
            file_names = list(filter(lambda f: f.endswith(".tar.gz"), file_names))
            new_dir_names = [x.replace('.tar.gz','_dir') for x in file_names]
            for dir_name in new_dir_names:
                path = hp_date_path+name+'/'+dir_name
                os.makedirs(path, exist_ok=True)
                paths.append(path)
        
paths

100%|██████████| 11/11 [00:00<00:00, 420.87it/s]


['../data_hdd4t/中華電信/honeypot/2020-01-10/cowrie/cowrie_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/cowrie/cowrie_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/amun/amun_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/amun/amun_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/glastopf/glastopf_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/glastopf/glastopf_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/dionaea/dionaea_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/dionaea/dionaea_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/cowrie/cowrie_log_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/cowrie/cowrie_file_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/amun/amun_log_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/amun/amun_file_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/glastopf/glastopf_file_2020-01-11_dir',
 '../

In [23]:
%%time
pool_result = []
pool = Pool() #processes=21
for path in paths:
    r = pool.apply_async(parallel_untar,(path,))
    pool_result.append(r)
pool.close()
pool.join()
for r in pool_result:
    print('return:',r)

!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4390>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4c90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4290>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4490>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4690>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4850>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4310>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163aa90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163ab90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4610>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163a750>
return: <multiprocessing.poo

'2020-01-11'

In [49]:


total, used, free = shutil.disk_usage(paths[0])

print("Total: %d GiB" % (total // (2**30)))
print("Used: %d GiB" % (used // (2**30)))
print("Free: %d GiB" % (free // (2**30)))

Total: 3667 GiB
Used: 2367 GiB
Free: 1113 GiB


In [34]:
if 1<2 and (2<3:
    print(1)

1


In [12]:
os.makedirs(pcap_path+new_dir_names[0], exist_ok=True)

In [5]:
!ls  ../

anaconda3  data  data_hdd4t  Download  Leo_code


In [25]:
cmd = ["ls",'../']
p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
out, err = p.communicate()
print(str(out))

b'anaconda3\ndata\ndata_hdd4t\nDownload\nLeo_code\n'


b''

spark test